In [ ]:
import os
import re
import time
import json
import sqlite3
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

In [ ]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
DB_PATH = "nova_support.db"

In [ ]:
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS tickets (
            ticket_id TEXT PRIMARY KEY,
            name TEXT,
            company TEXT,
            email TEXT,
            issue TEXT,
            priority TEXT,
            status TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

In [ ]:
def new_ticket_id():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM tickets")
    count = cur.fetchone()[0]
    conn.close()
    return f"RT-{1001 + count}"

In [ ]:
def create_ticket(name, company, email, issue, priority="P3"):
    tid = new_ticket_id()
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO tickets (ticket_id, name, company, email, issue, priority, status, created_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (tid, name, company, email, issue, priority.upper(), "OPEN", ts))
    conn.commit()
    conn.close()
    return tid, ts

In [ ]:
def get_ticket(ticket_id):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT * FROM tickets WHERE ticket_id=?", (ticket_id,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return None
    keys = ["ticket_id", "name", "company", "email", "issue", "priority", "status", "created_at"]
    return dict(zip(keys, row))

In [ ]:
def synthesize_speech(text):
    if not text.strip():
        return None
    output_path = Path(tempfile.gettempdir()) / "nova_reply.mp3"
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=text
    ) as response:
        response.stream_to_file(output_path)
    return str(output_path)

In [ ]:
SYSTEM_PROMPT = """
You are Nova, the AI Support and Sales Assistant for Reallytics.ai.
You help customers with:
- Reporting issues (create tickets)
- Checking existing tickets
- Providing product/service information
- Explaining pricing ranges
- Reassuring integration compatibility with client systems
Respond in a professional, business tone.
"""

In [ ]:
def detect_intent(message):
    text = message.lower()
    if any(k in text for k in ["create ticket", "open ticket", "new ticket", "issue", "problem"]):
        return "create_ticket"
    if re.search(r"rt-\d+", text):
        return "check_ticket"
    if "price" in text or "cost" in text:
        return "pricing"
    if "integration" in text:
        return "integration"
    return "general"

In [ ]:
def chat(message, history, model, name, company, email):
    history_msgs = [{"role": h["role"], "content": h["content"]} for h in history]
    intent = detect_intent(message)

    if intent == "create_ticket":
        priority = "P2" if "urgent" in message.lower() or "high" in message.lower() else "P3"
        tid, ts = create_ticket(name, company, email, message, priority)
        text = f"A new support ticket has been created.\nTicket ID: {tid}\nCreated at: {ts}\nStatus: OPEN"
        yield text, synthesize_speech(text)
        return

    if intent == "check_ticket":
        match = re.search(r"(rt-\d+)", message.lower())
        if match:
            ticket_id = match.group(1).upper()
            data = get_ticket(ticket_id)
            if data:
                text = (
                    f"Ticket {ticket_id} Details:\n"
                    f"Issue: {data['issue']}\n"
                    f"Status: {data['status']}\n"
                    f"Priority: {data['priority']}\n"
                    f"Created at: {data['created_at']}"
                )
            else:
                text = f"No ticket found with ID {ticket_id}."
        else:
            text = "Please provide a valid ticket ID."
        yield text, synthesize_speech(text)
        return

In [ ]:
def chat(message, history, model, name, company, email):
    if not message.strip():
        yield "Please type a message to start.", None
        return

    history_msgs = [{"role": h["role"], "content": h["content"]} for h in history]
    intent = detect_intent(message)
    reply, audio_path = "", None

    if intent == "create_ticket":
        priority = "P2" if "urgent" in message.lower() or "high" in message.lower() else "P3"
        tid, ts = create_ticket(name, company, email, message, priority)
        reply = f"A new support ticket has been created.\nTicket ID: {tid}\nCreated at: {ts}\nStatus: OPEN"
        audio_path = synthesize_speech(reply)
        yield reply, audio_path
        return

    if intent == "check_ticket":
        match = re.search(r"(rt-\d+)", message.lower())
        if match:
            ticket_id = match.group(1).upper()
            data = get_ticket(ticket_id)
            if data:
                reply = (
                    f"Ticket {ticket_id} Details:\n"
                    f"Issue: {data['issue']}\n"
                    f"Status: {data['status']}\n"
                    f"Priority: {data['priority']}\n"
                    f"Created at: {data['created_at']}"
                )
            else:
                reply = f"No ticket found with ID {ticket_id}."
        else:
            reply = "Please provide a valid ticket ID."
        audio_path = synthesize_speech(reply)
        yield reply, audio_path
        return

    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history_msgs + [{"role": "user", "content": message}]
    stream = client.chat.completions.create(model=model, messages=messages, stream=True)

    full_reply = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        full_reply += delta
        yield full_reply, None  
    audio_path = synthesize_speech(full_reply)
    yield full_reply, audio_path 

In [ ]:
init_db()

In [ ]:
with gr.Blocks(title="Nova | Business AI Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Nova | Reallytics.ai Customer Support & Sales Assistant")
    gr.Markdown(
        "Nova helps clients create or track support tickets, understand services, and explore automation options. "
        "Type your questions and Nova will respond in both text and voice."
    )

    with gr.Row():
        name = gr.Textbox(label="Your Name", placeholder="Liam")
        company = gr.Textbox(label="Company (optional)", placeholder="ABC Corp")
    email = gr.Textbox(label="Email", placeholder="you@example.com")

    model = gr.Dropdown(["gpt-4o-mini", "gpt-4", "gpt-3.5-turbo"], value="gpt-4o-mini", label="Model")

    audio_output = gr.Audio(label="Nova's Voice Reply", autoplay=True, interactive=False)

    gr.ChatInterface(
        fn=chat,
        type="messages",
        additional_inputs=[model, name, company, email],
        additional_outputs=[audio_output],
        title="Chat with Nova",
        description="Ask about tickets, automation services, pricing, or integration and Nova will also speak her reply."
    )

if __name__ == "__main__":
    demo.launch()